In [1]:
import torch
import os
from os.path import exists
import torch.nn as nn
from torch.nn.functional import log_softmax, pad, one_hot
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
from torch.utils.data import DataLoader
import random
import json
import csv
### utils.py ###

class Dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
# end



# Takes the file paths as arguments
def parse_csv_file_to_json(path_file_csv):
    # create a dictionary
    elements = []

    # Open a csv reader called DictReader
    with open(path_file_csv, encoding='utf-8') as file_csv:
    #with open(path_file_csv) as file_csv:
        reader_csv = csv.DictReader(file_csv, delimiter="\t")

        # Convert each row into a dictionary
        # and add it to data
        for dict_head_value in reader_csv:
            element = {}

            for head, value in dict_head_value.items():
                #print(value)
                if value and (value[0] in ["[", "{"]):
                    #element[head] = eval(value)
                    element[head] = value
                else:
                    element[head] = value

            elements.append(element)
        # end
    # end

    return elements
# end

### utils.py ###




### core.py ###

"Produce N identical layers."
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
# end



class MultiHeadedAttention(nn.Module):

    "Take in model size and number of heads."
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
    # end


    "Compute 'Scaled Dot Product Attention'"
    def attention(self, query, key, value, mask=None, dropout=None):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # print('jinyuj: scores: {}, mask: {}'.format(scores.shape, mask.shape))
            scores = scores.masked_fill(mask == 0, -1e9)
        # end
        p_attn = scores.softmax(dim=-1)
        if dropout is not None:
            p_attn = dropout(p_attn)
        # end
        return torch.matmul(p_attn, value), p_attn
    # end


    "Implements Figure 2"
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = self.attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)
    # end
# end class


"""
A residual connection followed by a layer norm.
Note for code simplicity the norm is first as opposed to last.
"""
class ResidualLayer(nn.Module):

    def __init__(self, size, dropout=0.1, eps=1e-6):
        super(ResidualLayer, self).__init__()
        self.norm = torch.nn.LayerNorm(size, eps)
        self.dropout = nn.Dropout(dropout)
    # end

    "Apply residual connection to any sublayer with the same size."
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))
    # end
# end class


class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))
    # end
# end


class SimpleIDEmbeddings(nn.Module):
    def __init__(self, size_vocab, dim_hidden, id_pad):
        super(SimpleIDEmbeddings, self).__init__()
        self.lut = nn.Embedding(size_vocab, dim_hidden, padding_idx=id_pad)
        self.dim_hidden = dim_hidden

    def forward(self, x):
        result = self.lut(x)
        return result * math.sqrt(self.dim_hidden)
    # end

    def get_shape(self):
        return (self.lut.num_embeddings, self.lut.embedding_dim)
    # end
# end


"Implement the PE function."
class PositionalEncoding(nn.Module):

    def __init__(self, dim_positional, max_len=512):
        super(PositionalEncoding, self).__init__()

        # Compute the positional encodings once in log space.
        self.dim_positional = dim_positional
        pe = torch.zeros(max_len, dim_positional)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, dim_positional, 2) * -(math.log(10000.0) / dim_positional)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).to('cuda')
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return x
    # end
# end


class SimpleEmbedder(nn.Module):    # no segment embedder as we do not need that
    def __init__(self, size_vocab=None, dim_hidden=128, dropout=0.1, id_pad=0):
        super(SimpleEmbedder, self).__init__()
        self.size_vocab = size_vocab
        self.dim_hidden = dim_hidden
        self.id_pad = id_pad

        self.embedder = nn.Sequential(
            SimpleIDEmbeddings(size_vocab, dim_hidden, id_pad),
            PositionalEncoding(dim_hidden),
            nn.Dropout(dropout)
        )
    # end

    def forward(self, ids_input):   # (batch, seqs_with_padding)
        return self.embedder(ids_input)
    # end

    def get_vocab_size(self):
        return self.size_vocab
    # end
# end

### core.py ###



class SimpleEncoderLayer(nn.Module):

    def __init__(self, dim_hidden, dim_feedforward, n_head, dropout=0.1):
        super(SimpleEncoderLayer, self).__init__()

        self.n_head = n_head
        self.dim_hidden = dim_hidden
        self.dim_feedforward = dim_feedforward

        self.layer_attention = MultiHeadedAttention(n_head, dim_hidden)
        self.layer_feedforward = PositionwiseFeedForward(dim_hidden, dim_feedforward, dropout)
        self.layers_residual = clones(ResidualLayer(dim_hidden, dropout), 2)
    # end

    def forward(self, embeddings, masks, *args):
        embeddings = self.layers_residual[0](embeddings, lambda embeddings: self.layer_attention(embeddings, embeddings, embeddings, masks))
        return self.layers_residual[1](embeddings, self.layer_feedforward)
    # end
# end



class SimpleDecoderLayer(nn.Module):

    def __init__(self, dim_hidden, dim_feedforward, n_head, dropout=0.1):
        super(SimpleDecoderLayer, self).__init__()

        self.n_head = n_head
        self.dim_hidden = dim_hidden
        self.dim_feedforward = dim_feedforward

        self.layer_attention_decoder = MultiHeadedAttention(n_head, dim_hidden)
        self.layer_attention_encoder = MultiHeadedAttention(n_head, dim_hidden)
        self.layer_feedforward = PositionwiseFeedForward(dim_hidden, dim_feedforward, dropout)
        self.layers_residual = clones(ResidualLayer(dim_hidden, dropout), 3)

    def forward(self, embeddings, masks_encoder, output_encoder, masks_decoder, *args):
        embeddings = self.layers_residual[0](embeddings, lambda embeddings: self.layer_attention_decoder(embeddings, embeddings, embeddings, masks_decoder))
        embeddings = self.layers_residual[1](embeddings, lambda embeddings: self.layer_attention_encoder(embeddings, output_encoder, output_encoder, masks_encoder))
        return self.layers_residual[2](embeddings, self.layer_feedforward)
    # end
# end


class SimpleTransformerStack(nn.Module):

    def __init__(self, obj_layer, n_layers):
        super(SimpleTransformerStack, self).__init__()
        self.layers = clones(obj_layer, n_layers)

        self.norm = torch.nn.LayerNorm(obj_layer.dim_hidden)
        self.keys_cache = ['output']
        self.cache = Dotdict({
            'output': None
        })
    # end

    def forward(self, embedding_encoder=None, masks_encoder=None, output_encoder=None, embedding_decoder=None, masks_decoder=None ,noncache=False, **kwargs):  # input -> (batch, len_seq, vocab)

        if output_encoder is not None and embedding_decoder is not None and masks_decoder is not None:
            embeddings = embedding_decoder
        else:
            embeddings = embedding_encoder
        # end

        for layer in self.layers:
            embeddings = layer(embeddings, masks_encoder, output_encoder, masks_decoder)
        # end

        output = self.norm(embeddings)

        if not noncache:
            self.cache.output = output
        # end

        return output
    # end

    # def get_vocab_size(self):
    #     return self.embedder.embedder_token.shape[-1]
    # # end

    def clear_cache(self):
        for key_cache in self.keys_cache:
            self.cache[key_cache] = None
        # end
    # end
# end


class SimpleEncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, embedder_encoder, embedder_decoder, pooling=False):
        super(SimpleEncoderDecoder, self).__init__()

        self.pooling = pooling
        
        self.embedder_encoder = embedder_encoder
        self.encoder = encoder

        self.embedder_decoder = embedder_decoder
        self.decoder = decoder
        
        self.keys_cache = ['output_encoder_pooled']
        self.cache = Dotdict({
            'output_encoder_pooled': None
        })
    # end

    def forward(self, ids_encoder=None, masks_encoder=None, ids_decoder=None, masks_decoder=None, nocache=False, **kwargs):
        
        output_encoder = self.embed_and_encode(ids_encoder=ids_encoder, masks_encoder=masks_encoder, nocache=nocache)
        output = output_encoder
        
        if self.pooling:
            output_encoder_refilled = output_encoder.masked_fill(masks_encoder.transpose(-1,-2)==False, 0)
            output_encoder_pooled = torch.mean(output_encoder_refilled, dim=-2)
            self.cache.output_encoder_pooled = output_encoder_pooled
            
            output_encoder_pooled_expanded = output_encoder_pooled.unsqueeze(-2).expand(output_encoder.shape)
            output = output_encoder_pooled_expanded
        # end
        
        if self.embedder_decoder and self.decoder:
            output_decoder = self.embed_and_decode(ids_decoder=ids_decoder, masks_encoder=masks_encoder, output_encoder=output, masks_decoder=masks_decoder, nocache=nocache)
            output = output_decoder
        # end if
        
        return output
    # end
    
    def embed_and_encode(self, ids_encoder=None, masks_encoder=None, nocache=False, **kwargs):
        self.encoder.clear_cache()
        
        embedding_encoder = self.embedder_encoder(ids_encoder)
        output_encoder = self.encoder(
            embedding_encoder=embedding_encoder,
            masks_encoder=masks_encoder,
            nocache=nocache
        )
        
        return output_encoder
    # end

    
    def embed_and_decode(self, ids_decoder=None, masks_encoder=None, output_encoder=None, masks_decoder=None, nocache=False, **kwargs):
        self.decoder.clear_cache()
        
        embedding_decoder = self.embedder_decoder(ids_decoder)
        output_decoder = self.decoder(
            masks_encoder=masks_encoder,
            output_encoder=output_encoder,    #(len_seq, dim_hidden) -> (1, dim_hidden)
            embedding_decoder=embedding_decoder,
            masks_decoder=masks_decoder,
            nocache=nocache
        )

        return output_decoder
    # end
    

    def clear_cache(self):
        self.encoder.clear_cache()
        
        for key_cache in self.keys_cache:
            self.cache[key_cache] = None
        # end
        
        if self.decoder:
            self.decoder.clear_cache()
        # end
    # end


    def get_vocab_size(self, name_embedder):
        embedder = getattr(self, f'embedder_{name_embedder}')
        return embedder.get_vocab_size()
    # end

# end

class LinearAndNorm(nn.Module):
    def __init__(self, dim_in = None, dim_out = None, eps_norm=1e-12):
        super(LinearAndNorm, self).__init__()

        self.linear = torch.nn.Linear(dim_in, dim_out)
        self.norm = torch.nn.LayerNorm(dim_out, eps_norm)
    # end

    def forward(self, seqs_in):
        return self.norm(self.linear(seqs_in).relu())
    # end
# end




class TokenizerWrapper:
    def __init__(self, vocab, splitter):
        self.splitter = splitter
        self.vocab = vocab

        self.id_pad = len(vocab)
        self.id_cls = len(vocab) + 1
        self.id_sep = len(vocab) + 2
        self.id_mask = len(vocab) + 3
        
        self.size_vocab = len(vocab) + 4

        self.token_pad = '[P@D]'
        self.token_cls = '[CL$]'
        self.token_sep = '[$EP]'
        self.token_mask = '[M@$K]'
           
        self.index_id_token_special = {
            self.id_pad: self.token_pad,
            self.id_cls: self.token_cls,
            self.id_sep: self.token_sep,
            self.id_mask: self.token_mask
        }
        
    # end

    def encode(self, line):
        return self.vocab([doc.text.lower() for doc in self.splitter(line)])
    # end

    def decode(self, tokens):
        words = []
        for token in tokens:
            token = int(token)
            
            if token in self.index_id_token_special:
                word_target = self.index_id_token_special[token]
            else:
                try:
                    word_target = vocab.lookup_token(token)
                except:
                    word_target = '[ERROR_LOOKUP_{}]'.format(token)
                # end
            # end
            
            words.append(word_target)
        # end
        
        return ' '.join(words)
    # end
# end



class Batch:
    DEVICE = 'cuda'

    def __init__(self, **kwargs):
        self.kwargs = {}
        for k, v in kwargs.items():
            if v is not None and type(v) is not bool:
                self.kwargs[k] = v.to(Batch.DEVICE)
        # end
    # end

    def __call__(self):
        return self.kwargs
    # end
# end



class Collator_S2S:

    def __init__(self, tokenizer, size_seq_max, need_masked=0.3):
        self.tokenizer = tokenizer
        self.size_seq_max = size_seq_max
        self.need_masked = need_masked
    # end
    

    def __call__(self, list_corpus_source):

        tokens_input_encoder = []
        tokens_input_decoder = []
        tokens_label_decoder = []
        labels_similarity = []

        for corpus_source in list_corpus_source: # (line0, line1, sim), output of zip remove single case
            if len(corpus_source) == 3:
                corpus_line = [corpus_source[0], corpus_source[1]]
                labels_similarity.append(corpus_source[2])
            else:
                corpus_line = [corpus_source[1]]
            # end
            
            for line in corpus_line:
                tokens = self.tokenizer.encode(line)

                # TODO: check edge
                if len(tokens) > self.size_seq_max - 2:
                    tokens = tokens[:self.size_seq_max-2]
                # end

                tokens_input_encoder.append([self.tokenizer.id_cls] + tokens + [self.tokenizer.id_sep])
                tokens_input_decoder.append([self.tokenizer.id_cls] + tokens)
                tokens_label_decoder.append(tokens + [self.tokenizer.id_sep])
            # end
            

        # end

        inputs_encoder, masks_encoder, segments_encoder, labels_encoder = self.pad_sequences(tokens_input_encoder, self.size_seq_max, need_masked=self.need_masked)
        inputs_decoder, masks_decoder, segments_decoder, _ = self.pad_sequences(tokens_input_decoder, self.size_seq_max, need_diagonal=True)
        labels_decoder, masks_label, segments_label, _ = self.pad_sequences(tokens_label_decoder, self.size_seq_max)
        # labels_similarity = torch.Tensor(labels_similarity).unsqueeze(0).transpose(0,1)
        labels_similarity = torch.Tensor(labels_similarity)

        return Batch(
            ids_encoder=inputs_encoder,  # contains [mask]s
            masks_encoder=masks_encoder,
            labels_encoder=labels_encoder,  # doesn't contain [mask]
            segments_encoder=segments_encoder,
            ids_decoder=inputs_decoder,
            masks_decoder=masks_decoder,
            labels_decoder=labels_decoder,
            segments_label=segments_label,
            labels_similarity=labels_similarity
        )
    # end

    
    # return masks_attention?, return masks_segment?
    def pad_sequences(self, sequences, size_seq_max, need_diagonal=False, need_masked=0): # need_diagonal and need_masked cannot both set, one for bert seq one for s2s seq
        id_pad = self.tokenizer.id_pad
        id_mask = self.tokenizer.id_mask

        sequences_padded = []
        sequences_masked_padded = []

        for sequence in sequences:
            len_seq = len(sequence)

            count_pad = size_seq_max - len_seq

            sequence = torch.LongTensor(sequence)
            sequence_padded = torch.cat((sequence, torch.LongTensor([id_pad] * count_pad)))
            sequences_padded.append(sequence_padded)

            if need_masked:
                index_masked = list(range(1, len_seq-1))
                random.shuffle(index_masked)
                index_masked = torch.LongTensor(index_masked[:int(need_masked * (len_seq-2))])

                sequence_masked = sequence.detach().clone()
                sequence_masked.index_fill_(0, index_masked, id_mask)
                sequence_masked_padded = torch.cat((sequence_masked, torch.LongTensor([id_pad] * count_pad)))
                
                sequences_masked_padded.append(sequence_masked_padded)
            # end
    #   # end for

        inputs = torch.stack(sequences_padded)  # (batch, size_seq_max)
        if need_masked:
            inputs_masked_padded = torch.stack(sequences_masked_padded)
        # end

        masks_segment = (inputs != self.tokenizer.id_pad).unsqueeze(-2)    #(nbatch, 1, seq)
        masks_attention = self.make_std_mask(inputs, self.tokenizer.id_pad) if need_diagonal else masks_segment

        if need_masked:
            masks_masked = (inputs_masked_padded != id_mask).unsqueeze(-2)
            masks_attention = masks_attention & masks_masked
            return inputs_masked_padded, masks_attention, masks_segment, inputs # (inputs, masks_attention, masks_segment, labels)
        else:
            return inputs, masks_attention, masks_segment, None
        # end
    # end


    def subsequent_mask(self, size):
        "Mask out subsequent positions."
        attn_shape = (1, size, size)
        subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
            torch.uint8
        )
        return subsequent_mask == 0

    def make_std_mask(self, tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & self.subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask
    # end
# end

In [2]:
import spacy


def Multi30k(language_pair=None):
    corpus_lines_train = []

    for lan in language_pair:
        with open('text/train.{}'.format(lan), 'r') as file:
            corpus_lines_train.append(file.read().splitlines())
        # end
    # end

    corpus_train = list(zip(*corpus_lines_train))

    corpus_lines_eval = []

    for lan in language_pair:
        with open('text/val.{}'.format(lan), 'r') as file:
            corpus_lines_eval.append(file.read().splitlines())
        # end
    # end

    corpus_eval = list(zip(*corpus_lines_eval))

    return corpus_train, corpus_eval, None
# end


def Quora(split=0.05):
    filename_quora = 'quora_duplicate_questions.tsv'
    
    contents_quora = parse_csv_file_to_json(filename_quora)
    list_corpus_quora = []
    for c in contents_quora:
        label = int(c['is_duplicate'])
        score = 1.0 if label else 0.5
        corpus_quora = (c['question1'], c['question2'], score)
        list_corpus_quora.append(corpus_quora)
    # end
    
    indexs_all = list(range(len(list_corpus_quora)))
    random.shuffle(indexs_all)
    
    index_split = int(split * len(list_corpus_quora))
    
    indexs_eval = indexs_all[:index_split]
    indexs_train = indexs_all[index_split:]
    
    list_corpus_eval = [list_corpus_quora[i_e] for i_e in indexs_eval]
    list_corpus_train = [list_corpus_quora[i_t] for i_t in indexs_train]
    
    return list_corpus_train, list_corpus_eval, None
# end


def load_vocab(filename_vocab):
    vocab_tgt = torch.load(filename_vocab)
    return vocab_tgt
# end

def load_spacy():
    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_en
# end

In [3]:
class SimpleEncoderHead_MLM(nn.Module):

    def __init__(self, model, size_vocab, dim_hidden=128):
        super(SimpleEncoderHead_MLM, self).__init__()
        
        self.ffn = LinearAndNorm(dim_in=dim_hidden, dim_out=dim_hidden)
        self.extractor = torch.nn.Linear(dim_hidden, size_vocab, bias=False)
        self.extractor.weight = nn.Parameter(model.embedder_encoder.embedder[0].lut.weight)
        
        self.keys_cache = ['labels_mlm', 'masks_encoder', 'segments_encoder', 'output']
        self.cache = Dotdict({
            'labels_mlm': None,
            'masks_encoder': None,
            'segments_encoder': None,
            'output': None
        })
        
        self.func_loss = torch.nn.CrossEntropyLoss()
    # end


    def forward(self, model, labels_encoder=None, segments_encoder=None, masks_encoder=None, nocache=False, **kwargs):   # labels_input -> (batch, seq, labels)
        output_encoder = model.encoder.cache.output
        output_ffn = self.ffn(output_encoder)
        output_mlm = self.extractor(output_ffn) # output_mlm = prediction_logits

        if not nocache:
            self.cache.labels_mlm = labels_encoder
            self.cache.masks_encoder = masks_encoder
            self.cache.segments_encoder = segments_encoder
            self.cache.output = output_mlm
        # end

        return output_mlm
    # end
    
    def get_loss(self):
        
        labels_mlm = self.cache.labels_mlm
        masks_encoder = self.cache.masks_encoder
        segments_encoder = self.cache.segments_encoder
        output_mlm = self.cache.output
        
        segments_encoder_2d = segments_encoder.transpose(-1,-2)[:,:,0]
        loss_segments = self.func_loss(output_mlm.masked_select(segments_encoder_2d.unsqueeze(-1)).reshape(-1, output_mlm.shape[-1]), labels_mlm.masked_select(segments_encoder_2d))
        
        masks_masked = torch.logical_xor(masks_encoder, segments_encoder) & segments_encoder # True is masked
        masks_masked_perbatch = masks_masked[:,0,:]
        loss_masked = self.func_loss(output_mlm.masked_select(masks_masked_perbatch.unsqueeze(-1)).reshape(-1, output_mlm.shape[-1]), labels_mlm.masked_select(masks_masked_perbatch))       
        
        loss_mlm = loss_segments + loss_masked * 3
        return loss_mlm
    # end

    
    def clear_cache(self):
        for key_cache in self.keys_cache:
            self.cache[key_cache] = None
        # end
    # end

# end

In [4]:
class SimpleDecoderHead_S2S(nn.Module):

    def __init__(self, model, size_vocab, dim_hidden=128):
        super(SimpleDecoderHead_S2S, self).__init__()
        
        self.ffn = LinearAndNorm(dim_in=dim_hidden, dim_out=dim_hidden)
        self.extractor = torch.nn.Linear(dim_hidden, size_vocab, bias=False)
        self.extractor.weight = nn.Parameter(model.embedder_decoder.embedder[0].lut.weight)

        self.func_loss = torch.nn.CrossEntropyLoss()
        
        self.keys_cache = ['output', 'labels_s2s', 'segments_decoder']
        self.cache = Dotdict({
            'output': None,
            'labels_s2s': None,
            'segments_decoder': None
        })

    # end



    def forward(self, model, labels_decoder=None, segments_label=None, nocache=False, **kwargs):   # labels_input -> (batch, seq, labels)
        output_decoder = model.decoder.cache.output
        output_ffn = self.ffn(output_decoder)
        output_s2s = self.extractor(output_ffn)   # output_mlm = prediction_logits
        
        if not nocache:
            self.cache.segments_label = segments_label
            self.cache.labels_s2s =  labels_decoder
            self.cache.output = output_s2s
        # end

        return output_s2s
    # end


    def get_loss(self):
        labels_s2s = self.cache.labels_s2s
        output_s2s = self.cache.output
        
        segments_label = self.cache.segments_label
        segments_label_2d = segments_label.transpose(-1,-2)[:,:,0]

        loss_segments = self.func_loss(output_s2s.masked_select(segments_label_2d.unsqueeze(-1)).reshape(-1, output_s2s.shape[-1]), labels_s2s.masked_select(segments_label_2d))
        return loss_segments * 4
    # end


    def evaluate(self):
        pass
    # end


    def clear_cache(self):
        for key_cache in self.keys_cache:
            self.cache[key_cache] = None
        # end
    # end
# end

In [5]:
class SimpleEncoderHead_Similarity(nn.Module):

    def __init__(self):
        super(SimpleEncoderHead_Similarity, self).__init__()

        self.func_loss = torch.nn.MSELoss()
        self.cos_score_transformation = torch.nn.Identity()
        self.keys_cache = ['labels_sim', 'output']
        self.cache = Dotdict({
            'labels_sim': None,
            'output': None
        })
    # end

    def forward(self, model, labels_similarity=None, nocache=False, **kwargs):  # labels_sim (batch/2, 1)   for every two sentences, we have a label

        output_encoder_pooled = model.cache.output_encoder_pooled
        size_batch, dim_hidden = output_encoder_pooled.shape

        if size_batch % 2 != 0:
            raise Exception('sim calculation is not prepared as size_batch % 2 != 0')
        # end

        # pooling (batch, pair, dim_hidden)
        output_pooling = output_encoder_pooled.squeeze(1).view(-1, 2, dim_hidden)   # might cls + sep, but abandon now (as it's not easy to get sep for every batch, different location)
        output_pooling_x1 = output_pooling[:, 0, :]
        output_pooling_x2 = output_pooling[:, 1, :]
        sims = self.cos_score_transformation(torch.cosine_similarity(output_pooling_x1, output_pooling_x2))  # -> (batch, scores)

        if not nocache:
            self.cache.output = sims
            self.cache.labels_sim = labels_similarity
        # end

        return sims
    # end

    def get_loss(self):
        sims = self.cache.output
        labels_sim = self.cache.labels_sim

        loss_sim = self.func_loss(sims, labels_sim)
        return loss_sim * 64
    # end

    def clear_cache(self):
        for key_cache in self.keys_cache:
            self.cache[key_cache] = None
        # end
    # end

    def evaluate(self):
        pass
    # end
# end


In [6]:
class HeadManager(nn.Module):
    def __init__(self):
        super(HeadManager, self).__init__()
        self.index_name_head = set()
    # end

    def register(self, head):
        name_head = head.__class__.__name__
        setattr(self, name_head, head)
        self.index_name_head.add(name_head)
        return self
    # end

    def forward(self, model, **kwargs):
        for name in self.index_name_head:
            head = getattr(self, name)
            head.forward(model, **kwargs)
        # end
    # end

    def get_head(self, klass):
        return getattr(self, klass.__name__)
    # end

    def clear_cache(self):
        for name_head in self.index_name_head:
            getattr(self, name_head).clear_cache()
        # end
    # end
# end


class Trainer(nn.Module):
    def __init__(self, model=None, manager=None):
        super(Trainer, self).__init__()
        self.model = model
        self.manager = manager
    # end

    def forward(self, **kwargs):
        self.clear_cache()
        
        self.model.forward(**kwargs)
        self.manager.forward(self.model, **kwargs)
    # end
    
    def clear_cache(self):
        self.model.clear_cache() if self.model else None
        self.manager.clear_cache() if self.manager else None
    # end
# end

In [7]:
class Builder:
    
    @classmethod
    def build_model_with_mlm_v2(cls, size_vocab, dim_hidden, dim_feedforward, n_head, n_layer):
        embedder_encoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_encoder = SimpleEncoderLayer(dim_hidden, dim_feedforward, n_head)
        encoderstack = SimpleTransformerStack(sample_encoder, n_layer)

        model = SimpleEncoderDecoder(encoderstack, None, embedder_encoder, None)
        head_mlm = SimpleEncoderHead_MLM(model, size_vocab, dim_hidden)

        manager = HeadManager().register(head_mlm)
        trainer = Trainer(model=model, manager=manager)

        return trainer
    # end
    
    @classmethod
    def build_model_with_s2s_v2(cls, size_vocab, dim_hidden, dim_feedforward, n_head, n_layer):
        embedder_encoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_encoder = SimpleEncoderLayer(dim_hidden, dim_feedforward, n_head)
        encoderstack = SimpleTransformerStack(sample_encoder, n_layer)
        
        embedder_decoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_decoder = SimpleDecoderLayer(dim_hidden, dim_feedforward, n_head)
        decoderstack = SimpleTransformerStack(sample_decoder, n_layer)

        model = SimpleEncoderDecoder(encoderstack, decoderstack, embedder_encoder, embedder_decoder, pooling=True)
        head_s2s = SimpleDecoderHead_S2S(model, size_vocab, dim_hidden)
        
        manager = HeadManager().register(head_s2s)
        trainer = Trainer(model=model, manager=manager)

        return trainer
    # end
    
    @classmethod
    def build_model_with_2heads(cls, size_vocab, dim_hidden, dim_feedforward, n_head, n_layer):
        embedder_encoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_encoder = SimpleEncoderLayer(dim_hidden, dim_feedforward, n_head)
        encoderstack = SimpleTransformerStack(sample_encoder, n_layer)
        
        embedder_decoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_decoder = SimpleDecoderLayer(dim_hidden, dim_feedforward, n_head)
        decoderstack = SimpleTransformerStack(sample_decoder, n_layer)

        model = SimpleEncoderDecoder(encoderstack, decoderstack, embedder_encoder, embedder_decoder, pooling=True)
        head_s2s = SimpleDecoderHead_S2S(model, size_vocab, dim_hidden)
        head_mlm = SimpleEncoderHead_MLM(model, size_vocab, dim_hidden)
        
        manager = HeadManager().register(head_s2s).register(head_mlm)
        trainer = Trainer(model=model, manager=manager)

        return trainer
    # end
    
    @classmethod
    def build_model_with_sim_v2(cls, size_vocab, dim_hidden, dim_feedforward, n_head, n_layer):
        embedder_encoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_encoder = SimpleEncoderLayer(dim_hidden, dim_feedforward, n_head)
        encoderstack = SimpleTransformerStack(sample_encoder, n_layer)

        model = SimpleEncoderDecoder(encoderstack, None, embedder_encoder, None, pooling=True)
        head_sim = SimpleEncoderHead_Similarity()

        manager = HeadManager().register(head_sim)
        trainer = Trainer(model=model, manager=manager)

        return trainer
    # end

    @classmethod
    def build_model_with_3heads(cls, size_vocab, dim_hidden, dim_feedforward, n_head, n_layer):
        embedder_encoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_encoder = SimpleEncoderLayer(dim_hidden, dim_feedforward, n_head)
        encoderstack = SimpleTransformerStack(sample_encoder, n_layer)
        
        embedder_decoder = SimpleEmbedder(size_vocab=size_vocab, dim_hidden=dim_hidden)
        sample_decoder = SimpleDecoderLayer(dim_hidden, dim_feedforward, n_head)
        decoderstack = SimpleTransformerStack(sample_decoder, n_layer)

        model = SimpleEncoderDecoder(encoderstack, decoderstack, embedder_encoder, embedder_decoder, pooling=True)
        head_s2s = SimpleDecoderHead_S2S(model, size_vocab, dim_hidden)
        head_mlm = SimpleEncoderHead_MLM(model, size_vocab, dim_hidden)
        head_sim = SimpleEncoderHead_Similarity()
        
        manager = HeadManager().register(head_s2s).register(head_mlm).register(head_sim)
        trainer = Trainer(model=model, manager=manager)

        return trainer
    # end
# end

In [8]:
import json
import transformers
from torch.utils.data import DataLoader, Dataset
from torchtext.data.functional import to_map_style_dataset


gpu = 0
torch.cuda.set_device(gpu)

epochs = 8

# source
seq_max = 16
batch_size = 32


# model & head
dim_hidden = 512
dim_feedforward = 512
n_head = 8
n_layer = 8

# optimizer
lr_base_optimizer = 1e-4
betas_optimizer = (0.9, 0.999)
eps_optimizer = 1e-9

# scheduler
warmup = 200


filename_vocab = 'vocab_quora.pt'
spacy_en = load_spacy()
vocab = load_vocab(filename_vocab)
tokenizer = TokenizerWrapper(vocab, spacy_en)

# train_iter, valid_iter, _ = Multi30k(language_pair=("de", "en"))
# train_source = to_map_style_dataset(train_iter)

# with open('corpus.json', 'r') as file:
#     train_source = json.load(file)
# # end

train_source, valid_source, _ = Quora(split=0.05)


collator = Collator_S2S(tokenizer, seq_max)
dataloader_train = DataLoader(train_source, batch_size, shuffle=False, collate_fn=collator)

# trainer = Builder.build_model_with_mlm_v2(tokenizer.size_vocab, dim_hidden, dim_feedforward, n_head, n_layer)
# trainer = Builder.build_model_with_2heads(tokenizer.size_vocab, dim_hidden, dim_feedforward, n_head, n_layer)
# trainer = Builder.build_model_with_sim_v2(tokenizer.size_vocab, dim_hidden, dim_feedforward, n_head, n_layer)
trainer = Builder.build_model_with_3heads(tokenizer.size_vocab, dim_hidden, dim_feedforward, n_head, n_layer)

for p in trainer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    # end
# end

trainer = trainer.to('cuda')

optimizer = torch.optim.Adam(trainer.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
decayRate = 0.96
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

print()

In [9]:
def train_a_batch(batch, trainer, optimizer=None, scheduler=None):
    trainer.train()
    trainer.forward(**batch())
    
    loss_s2s = trainer.manager.get_head(SimpleDecoderHead_S2S).get_loss()
    loss_mlm = trainer.manager.get_head(SimpleEncoderHead_MLM).get_loss()
    loss_sim = trainer.manager.get_head(SimpleEncoderHead_Similarity).get_loss()

    # crossentropy loss
    
    # loss_all = loss_s2s * 5
    # loss_all = loss_mlm
    # loss_all = loss_sim
    # loss_all = (loss_s2s + loss_mlm) / 2
    # loss_all = (loss_s2s + loss_mlm + loss_sim) / 3
    loss_all = (loss_s2s + loss_mlm + loss_sim)
    loss_all_value = loss_all.item()
    
    # print(loss_all)
    loss_all.backward()

    
    if optimizer:
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
    # end
    
    if scheduler:
        scheduler.step()
    # end
    
    trainer.clear_cache()
    return loss_all_value
# end

In [10]:
from datetime import datetime


for e in range(epochs):
    losss_per_e = []
    for i, batch in enumerate(dataloader_train):
        loss_current = train_a_batch(batch, trainer, optimizer, None)
        losss_per_e.append(loss_current)
        if i % 100 == 0:
            print('Epoch: {} Batch: {}, loss: {}, rate: {}'.format(e, i, loss_current, optimizer.param_groups[0]['lr']))
        # end
    # end
    
    loss_average_per_e = sum(losss_per_e) / len(losss_per_e)
    print('[{}] Epoch: {} ends. Average loss: {}'.format(datetime.utcnow(), e, loss_average_per_e))
    
    if e % 2 == 0:
        lr_scheduler.step() # schedule per 2 epoch
    # end
# end

Epoch: 0 Batch: 0, loss: 82.66297912597656, rate: 0.0001
Epoch: 0 Batch: 100, loss: 51.15053939819336, rate: 0.0001
Epoch: 0 Batch: 200, loss: 48.073997497558594, rate: 0.0001
Epoch: 0 Batch: 300, loss: 45.51436996459961, rate: 0.0001
Epoch: 0 Batch: 400, loss: 43.37275695800781, rate: 0.0001
Epoch: 0 Batch: 500, loss: 43.038516998291016, rate: 0.0001
Epoch: 0 Batch: 600, loss: 42.962806701660156, rate: 0.0001
Epoch: 0 Batch: 700, loss: 41.990665435791016, rate: 0.0001
Epoch: 0 Batch: 800, loss: 42.703651428222656, rate: 0.0001
Epoch: 0 Batch: 900, loss: 38.13771057128906, rate: 0.0001
[2023-11-20 12:05:26.704828] Epoch: 0 ends. Average loss: 45.72149917221069
Epoch: 1 Batch: 0, loss: 41.64433288574219, rate: 9.6e-05
Epoch: 1 Batch: 100, loss: 38.45461654663086, rate: 9.6e-05
Epoch: 1 Batch: 200, loss: 37.9468879699707, rate: 9.6e-05
Epoch: 1 Batch: 300, loss: 38.837318420410156, rate: 9.6e-05
Epoch: 1 Batch: 400, loss: 38.43572235107422, rate: 9.6e-05
Epoch: 1 Batch: 500, loss: 38.415

In [11]:
trainer.eval()
print()

In [12]:
# For s2s head
def greedy_generate(model, head, tokenizer, collator, **kwargs):
    id_start = tokenizer.id_cls
    id_end = tokenizer.id_sep
    id_pad = tokenizer.id_pad
    size_seq_max = collator.size_seq_max

    ids_encoder_twin = kwargs['ids_encoder']
    masks_encoder_twin = kwargs['masks_encoder']
    
    ids_decoder_all = []
    
    for j in range(ids_encoder_twin.shape[0]):
        ids_encoder = ids_encoder_twin[j,].unsqueeze(0)
        masks_encoder = masks_encoder_twin[j,].unsqueeze(0)

        output_encoder = model.embed_and_encode(ids_encoder=ids_encoder, masks_encoder=masks_encoder)
        ids_decoder = torch.zeros(1, 1).fill_(id_start).type_as(ids_encoder.data)

        for i in range(size_seq_max - 1):
            masks_decoder = collator.subsequent_mask(ids_decoder.size(1)).type_as(ids_encoder.data)
            output_decoder = model.embed_and_decode(ids_decoder=ids_decoder, masks_encoder=masks_encoder, output_encoder=output_encoder, masks_decoder=masks_decoder)

            output_ffn = head.ffn(output_decoder)
            output_s2s = head.extractor(output_ffn)   # output_mlm = prediction_logits

            logits_nextword = torch.softmax(output_s2s[:, -1], dim=-1)  # mynote: select dim2=-1, remain=all; last is the next

            id_nextword = torch.argmax(logits_nextword, dim=-1)
            id_nextword = id_nextword.data[0]

            if id_nextword == id_end:
                break
            # end

            ids_decoder = torch.cat([ids_decoder, torch.zeros(1, 1).type_as(ids_encoder.data).fill_(id_nextword)], dim=1)
        # end
        
        ids_pad = torch.full((1, size_seq_max - ids_decoder.shape[-1]), id_pad).type_as(ids_decoder.data)
        
        ids_decoder_all.append(torch.cat([ids_decoder, ids_pad], dim=-1).squeeze(0))
    # end for 

    return torch.stack(ids_decoder_all)
# end

# eval_source = to_map_style_dataset(valid_iter)
dataloader_eval = DataLoader(valid_source, 1, shuffle=False, collate_fn=collator)

for i, batch in enumerate(dataloader_eval):
    info_batch = batch()
    result = greedy_generate(trainer.model, trainer.manager.get_head(SimpleDecoderHead_S2S), tokenizer, collator, **info_batch)
    
    
    result_cpu_list = result.cpu().tolist()
    labels_decoder_cpu_list = info_batch['labels_decoder'].cpu().tolist()
    
    for result_cpu, labels_decoder in zip(result_cpu_list, labels_decoder_cpu_list):
    
        sentence_predicted = tokenizer.decode(result_cpu).split(' [PAD]')[0]
        sentence_origin = tokenizer.decode(labels_decoder).split(' [PAD]')[0]
        
        print('source: {}\ntarget: {}\n\n'.format(sentence_origin, sentence_predicted))
    # end
    
    if i >= 5:
        break
# end 

source: i 'm 18 . what should i do ? [SEP]
target: [CLS] i am a job . what should i should i do i do i


source: what can i do when i 'm 18 ? [SEP]
target: [CLS] what should i do when i do when i do n't know ? i


source: what is quoran of the week ? [SEP]
target: [CLS] what of the of the top 10 ?


source: what is a quoran ? [SEP]
target: [CLS] what is ?


source: is there any presence of the illuminati in india ? do they influence the [SEP]
target: [CLS] is there the there in the world war between the world war between the


source: are there any religions that have not been influenced by the illuminati ? [SEP]
target: [CLS] are there any are there any that are there any other than there any


source: does facebook pay you for page likes in the same manner that youtube pays [SEP]
target: [CLS] does you have an <unk> in the same time to make the same time


source: why is organic reach in facebook a lot higher than paid reach , even [SEP]
target: [CLS] is a private in a car in a perso

In [13]:
def decode_output(out_mlm, masks_masked_prebatch, labels_mlm, ids_encoder, tokenizer):
    # print segments
    # sentence_predicts = tokenizer.decode(out_mlm.softmax(dim=-1).argmax(dim=-1).masked_select(segments_encoder[:, 0, :]).numpy().tolist())
    # sentence_labels = tokenizer.decode(labels_mlm.masked_select(segments_encoder[:, 0, :]).numpy().tolist())
    # sentence_inputs = tokenizer.decode(ids_encoder.masked_select(segments_encoder[:, 0, :]).numpy().tolist())

    # print masks
    sentence_predicts = tokenizer.decode(out_mlm.softmax(dim=-1).argmax(dim=-1).masked_select(masks_masked_perbatch).numpy().tolist())
    sentence_labels = tokenizer.decode(labels_mlm.masked_select(masks_masked_perbatch).numpy().tolist())
    sentence_inputs = tokenizer.decode(ids_encoder.masked_select(masks_masked_perbatch).numpy().tolist())


#     sentence_predicts = tokenizer.decode(out_mlm.softmax(dim=-1).argmax(dim=-1).numpy().tolist()[0])
#     sentence_labels = tokenizer.decode(labels_mlm.numpy().tolist()[0])
#     sentence_inputs = tokenizer.decode(ids_encoder.numpy().tolist()[0])

    predicts_masked = out_mlm.softmax(dim=-1).argmax(dim=-1).masked_select(masks_masked_perbatch)
    labels_masked = labels_mlm.masked_select(masks_masked_perbatch)

    acc = torch.count_nonzero(predicts_masked == labels_masked) / labels_masked.shape[0]
    # acc = torch.count_nonzero(out_mlm.softmax(dim=-1).argmax(dim=-1).view(-1) == labels_mlm.view(-1)) / labels_mlm.view(-1).shape[0]
    return acc, sentence_labels, sentence_inputs, sentence_predicts
# end


# eval_source = to_map_style_dataset(valid_iter)
dataloader_eval = DataLoader(valid_source, 1, shuffle=False, collate_fn=collator)
# dataloader_eval = DataLoader(train_source, 1, shuffle=False, collate_fn=collator)

for i, batch in enumerate(dataloader_eval):
    info_batch = batch()
    trainer.forward(**info_batch)
    
    head = trainer.manager.get_head(SimpleEncoderHead_MLM)
    out_mlm = head.cache.output
    loss_mlm = head.get_loss()
    
    out_mlm = out_mlm.cpu().detach()
    loss_mlm = loss_mlm.cpu().detach()
    labels_mlm = info_batch['labels_encoder'].cpu().detach()
    masks_encoder = info_batch['masks_encoder'].cpu().detach()
    segments_encoder = info_batch['segments_encoder'].cpu().detach()
    ids_encoder = info_batch['ids_encoder'].cpu().detach()
    
    
    masks_masked = torch.logical_xor(masks_encoder, segments_encoder) & segments_encoder # True is masked
    masks_masked_perbatch = masks_masked[:,0,:]
    
    for j in range(masks_masked_perbatch.shape[0]):
        acc, sentence_labels, sentence_inputs, sentence_predicts = decode_output(out_mlm[j,].unsqueeze(0), masks_masked_perbatch[j,].unsqueeze(0), labels_mlm[j,].unsqueeze(0), ids_encoder[j,].unsqueeze(0), tokenizer)
        print('loss: {}, acc: {}\nsource: {}\ninput: {}\npredict: {}\n\n'.format(loss_mlm.item(), acc, sentence_labels, sentence_inputs, sentence_predicts))
    
    if i >= 5:
        break
    # end
# end

loss: 5.2500104904174805, acc: 1.0
source: i what what i
input: [MASK] [MASK] [MASK] i
predict: i what what i


loss: 5.2500104904174805, acc: 0.25
source: what when when 'm
input: what [MASK] [MASK] [MASK]
predict: what if if have


loss: 20.781038284301758, acc: 0.3333333432674408
source: quoran week the
input: [MASK] [MASK] the
predict: the <unk> the


loss: 20.781038284301758, acc: 1.0
source: a [SEP] ?
input: a [SEP] [MASK]
predict: a [SEP] ?


loss: 14.277353286743164, acc: 0.4285714328289032
source: there do they influence any the influence
input: [MASK] [MASK] [MASK] [MASK] any the [MASK]
predict: there what is is any the is


loss: 14.277353286743164, acc: 0.7142857313156128
source: there the illuminati ? any have ?
input: there the illuminati [MASK] [MASK] [MASK] [MASK]
predict: there the world ? any are ?


loss: 22.83403205871582, acc: 0.0
source: facebook you for likes for manner youtube pays
input: [MASK] [MASK] [MASK] [MASK] [MASK] manner youtube pays
predict: it a a <un